<a href="https://colab.research.google.com/github/Gustavo-Ros/Simulaci-n-2/blob/main/Frecuentistas_vs_Bayesianos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DIFERENCIAS ENTRE ENFOQUE FRECUENTISTAS Y BAYESIANOS

La principal diferencia o desacuerdo entre el enfoque frecuentista y el enfoque bayesiano radica en como se defina la probabilidad:


*   **Enfoque frecuentista**: La probabilidad es definida como la frecuencia en la que ocurre un evento tras un gran número de repeticiones. La probabilidad solo tiene significado en terminos de un caso limitado de repetidas mediciones.

*   **Enfoque bayesiano**: La probabilidad se define sobre una suposicion o conocimiento inicial de un evento, la cual se actualiza y se vuelve más precisa conforme se obtiene más información.

En el artículo se muestran diferentes ejemplos y como se resuelven utilizando ambos enfoques:

## EJEMPLO 1.- MEDICIONES DE FLUJO DE FOTONES

Se apunta un telescopio al cielo y se observa la luz que proviene de una sola estrella. EL flujo de fotones es constante con el tiempo, es decir $F$ esta fija. Dado un número $i$ de mediciones que tienen un flujo $F_i$ y un error $e_i^2$, se desea estimar el valor real de $F$

### ENFOQUE FRECUENTISTA

Se utiliza la máxima verosimilitud , asumiendo que los errores $e_i^2$ son Gaussianos. El estimador que se utiliza es la media

$$ \hat{F}=\frac{\sum w_i F_i}{\sum w_i}; \hspace{4mm} w_i=1 / e_i^2$$

In [1]:
import random as rnd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
np.random.seed(2)            #Semilla para reproducibilidad
e=np.random.normal(30,3,50)
F=np.random.normal(1000,e)

In [6]:
w=1./e**2
F_hat=np.sum(w*F)/np.sum(w)     #Estimador F
sigma_F=w.sum()**-0.5           #Desviación estándar del estimador (error)
print("Estimación: " ,F_hat)
print("Error: ",sigma_F)

Estimación:  998.6496963757094
Error:  4.113743971231106


Finalmente, la estimación realizada es $$\hat{F}=999 \pm 4$$

### ENFOQUE BAYESIANO

Se utiliza el enfoque Bayesiano : $$P( F | D) = \frac{P(D|F)P(F)}{P(D)}$$
Donde


*   $P(F|D)$ : La probabilidad de los paramteros del problema segun los datos dados.
*   $P(D|F)$ : La probabilidad, que es proporcional a la probabilidad usada en el enoque frecuentista.
*   $P(F)$ : Que codifica lo que se sabia del problema antes de considerar los datos $D$
*   $P(D)$ :  La evidencia del modelo, que en la práctoca equivale a siples términos normalizados.



## EJEMPLO 2.-  EL JUEGO DE BILLAR DE BAYES

2 individuos juegan en una mesa de billar, hay pone en un marca en un sitio que los 2 jugadores descononcen, el juego consiste en que al tirar una bola (con la misma probabilidad de caer en cualquier lado de la mesa), si la bola cae del lado derecho de la marca, el primer jugador obtiene un punto, si cae del lado izquierdo, el segundo jugador obtiene un punto.

Después de 8 tiros, el jugador 1 tiene 5 puntos y el jugador 2 tiene 3 puntos. **¿Cuál es la probabilidad de que el jugador 2 obtenga 6 puntos y gane?**

### ENFOQUE FRECUENTISTA

Se utiliza l amáxima verosimilitud, la cual , segun los eventos del juego, es $$\hat{p}=5/8$$
Luego, la probabilidad de que el jugador obtenga 3 puntos seguidos y gane esta dada por $$P(B)=(1-\hat{p})^3=0.053$$

### ENFOQUE BAYESIANO

Se utiliza la marginalidad sobre la probabilidad de que la bola caiga del lado del derecho de la marca (que el jugador 1 obtenga un punto) para integrar la incertidumbre sobre  donde está la marca:

$$P(B|D)= \frac{\int_0^1(1-p)^6p^5dp}{\int_0^1(1-p)^3p^5dp}$$

In [7]:
from scipy.special import beta
P_B_D= beta(6+1,5+1)/ beta(3+1,5+1)   #Las integrales son ejemplos de la funcion beta
P_B_D

0.09090909090909091

Finalmente $$P(B|D) \approx 0.091$$

**Notar que el enfoque frecuentista no toma en cuenta la incertidumbre de la posición de la marca, en cambio, el enfoque Bayesiano si lo hace**

## EJEMPLO 3.-  CONFIANZA VS CREDIBILIDAD: LA EXPONENCIAL TRUNCADA DE JAYNE

Se observa el timepo que tienen los fallos de un dispositivo despupes de que inhibidor químico se agote. Se busca estimar el tiempo que transcurre mientras el equipos no presenta fallos.

### ENFOQUE FRECUENTISTA

Se calculan intervalos de confianza $\theta$  y la maedia muestral para estimar el tiempo en que el equipo no falla.

Sin emargo, divhos intervalos son $$CI(\theta)=(10.2,12.5)$$ los cuales son valores muy por encima de de $\theta \leq 10$  

### ENFOQUE GAUSSIANO

Se utiliza $P(F)$ y el teorema de Bayes para obtener una distribución. Los intervalos creibles son $$CR(\theta)=(9.00, 10.0)$$
los cuales presetan valores que respetan $\theta \leq 10$



## EJEMPLO 4.- ENFOQUE BAYESINAO EN PRACTICA: CADENA DE MARKOV MONTE CARLO


###MODELO LINEAL SIMPLE

Se busc aajustar una linea de tres parámetros (la intersección $\alpha$, la pendiente $\beta$ y la dispersión de datos $\sigma$) a un conjunto de datos con errores desconocidos.

Se tienen los datos $D= \lbrace x_i,y_i \rbrace$, el modelo es $$\hat{y}(x_i | \alpha,\beta)=\alpha+\beta x_i$$
yla probabilidad es el producto de la distribucioón Gaussiana para cada punto

$$L(D|α, β, σ)=(2\pi σ^2)^{-N/2} ∏_{I=1}^{N} exp \left[ \frac{-[y_i-\hat{y}(x_i|α,β)]^2}{2 σ^2} \right]$$

Se evalua este modelo en el siguiente código:


### ENFOQUE FRECUENTISTA

Se utiliza regresión lineal y la máxima verosimilitud para obtener los valores optimos de los 3 parámetros.

Se asume además, que los errores tienen una dsitribución normal.

Se define:
 * El vector parámetro $\theta=[α \;\; \beta]^{T}$
 * El vector respuesta $Y=[y_1\;y_2\;y_3\; ... y_N]^T$
 * Y la matriz de diseño :
 $$ X = \left[ \begin{matrix} 1 & 1 & 1 & ... & 1 \\ x_1 & x_2 &x_3 & ... & x_N \end{matrix} \right] ^T$$

 Además, la solución de la máxima verosimilitud $\hat{\theta}=(X^TX)^{-1}(X^TY)$

 El intervalo de confianza es una elispe cuyos parámetros están definidos por la matriz: $$Σ_{\hat{\theta}} \equiv \left[ \begin{matrix} \sigma_{\alpha}^2 & \sigma_{α β} \\ σ_{α β} & σ_{α}^2 \end{matrix} \right]=σ^2(M^TM)^{-1}$$

 En código:

In [9]:
np.random.seed(42)                        #Semilla para reproducibilidad
theta_true=(25,0.5)                       #Valores realesde los parametros theta
xdata=100*np.random.rand(20)              #Valores aleatorios
ydata=theta_true[0]+theta_true[1]*xdata   #Valores reales de y
ydata=np.random.normal(ydata,10)

In [10]:
x=np.vstack([np.ones_like(xdata),xdata]).T                  #Matriz de diseño
theta_hat=np.linalg.solve(np.dot(x.T,x),np.dot(x.T,ydata))  #Coef de regresión
y_hat=np.dot(x,theta_hat)                                   #Estimaciones de y
sigma_hat=np.std(ydata-y_hat)                               #Desviación estandar (error)
sigma=sigma_hat**2 * np.linalg.inv(np.dot(x,x.T))           #Matriz de theta

In [11]:
import statsmodels.api as sm
X=sm.add_constant(xdata)
result=sm.OLS(ydata,X).fit()              #Ajusta modelo de regresion OLS
sigma_hat=result.params                   #Parámetros estimados
Sigma=result.cov_params()                 #Matriz de covarianza de los parámetros
print(result.summary2())                  #Resumen del modelo


                 Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.683   
Dependent Variable: y                AIC:                147.7737
Date:               2024-10-12 05:11 BIC:                149.7651
No. Observations:   20               Log-Likelihood:     -71.887 
Df Model:           1                F-statistic:        41.97   
Df Residuals:       18               Prob (F-statistic): 4.30e-06
R-squared:          0.700            Scale:              86.157  
-------------------------------------------------------------------
            Coef.    Std.Err.     t      P>|t|     [0.025    0.975]
-------------------------------------------------------------------
const      24.6361     3.7871   6.5053   0.0000   16.6797   32.5924
x1          0.4483     0.0692   6.4782   0.0000    0.3029    0.5937
-----------------------------------------------------------------
Omnibus:              1.996        Durbin-Watson:           2.758
Prob(Omnibus):   

### ENFOQUE BAYESIANO

El enfoque Bayesiano se encapsula en $P(F|D)$ , que es proporcional producto de máxima veromilitud por $P(F)$.

Se utiliza el teorema de Bayes junto con el método de Cadenas de Markov Monte Cralo.

En código:

In [12]:
!pip install emcee

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.4 MB/s eta 0:00:00


In [14]:
import emcee # version 2.0
def log_prior(theta):                                #Funcion prior  P(F)
  alpha, beta, sigma = theta
  if sigma < 0:
    return -np.inf # log(0)
  else:
    return (-1.5 * np.log(1 + beta**2)- np.log(sigma))

def log_like(theta, x, y):                           #Funcion de verosimilitud
    alpha, beta, sigma = theta
    y_model = alpha + beta * x
    return -0.5 * np.sum(np.log(2*np.pi*sigma**2)+(y-y_model)**2 / sigma**2)

def log_posterior(theta, x, y):                      #Funcion posterior P(F|D)
  return log_prior(theta) + log_like(theta,x,y)


In [16]:
ndim = 3        #Parametros
nwalkers = 50   #Numero de caminantes
nburn = 1000    #Iteraciones
nsteps = 2000   #Numeor de pasos
starting_guesses = np.random.rand(nwalkers, ndim)

In [18]:
sampler = emcee.EnsembleSampler(nwalkers, ndim,log_posterior,args=[xdata,ydata])
sampler.run_mcmc(starting_guesses, nsteps)        #Muestrep MCMM
trace = sampler.chain[:, nburn:, :]
trace = trace.reshape(-1, ndim).T

In [19]:
import pymc as pm

In [20]:
import pymc as pm
basic_model = pm.Model()

with basic_model:
    #Funcion prior P(F) para los 3 parametros
    alpha = pm.Normal("alpha", mu=-100, sigma=100)
    beta = pm.Normal("beta", mu=0, sigma=10, shape=2)
    sigma = pm.HalfNormal("sigma", sigma=1)

    mu = alpha + beta[0] * xdata  #Valor esperado

    y = pm.Normal("Y_obs", mu=mu, sigma=sigma, observed=ydata)  #Verosimilitud

    y_model = pm.Deterministic("y_model", mu)


    model = dict(alpha=alpha, beta=beta, sigma=sigma,y_model=y_model, y=y)

    trace = pm.sample(10000, tune=5000, chains=4, cores=4)


Output()

In [21]:
!pip install stan

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for stan
  Running setup.py clean for stan
Failed to build stan
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (stan)


In [29]:
!pip install pystan
import pystan
model_code = """
data {
int<lower=0> N; // number of points
real x[N]; // x values
real y[N]; // y values
}
parameters {
real alpha_perp;
real<lower=-pi()/2, upper=pi()/2> theta;
real log_sigma;
}
transformed parameters {
real alpha;
real beta;
real sigma;
real ymodel[N];
alpha <- alpha_perp / cos(theta);
beta <- sin(theta);
sigma <- exp(log_sigma);
for (j in 1:N)
ymodel[j] <- alpha + beta * x[j];
}
model {
y ~ normal(ymodel, sigma);
}
"""
# perform the fit & extract traces
data = {'N': len(xdata), 'x': xdata, 'y': ydata}
fit = pystan.stan(model_code=model_code, data=data,iter=25000, chains=4)
tr = fit.extract()
trace = [tr['alpha'], tr['beta'], tr['sigma']]


ModuleNotFoundError: No module named 'pystan'